##### The cell below is for you to keep track of the libraries used and install those libraries quickly
##### Ensure that the proper library names are used and the syntax of `%pip install PACKAGE_NAME` is followed

In [ ]:
#%pip install pandas
#%pip install matplotlib
# add commented pip installation lines for packages used as shown above for ease of testing
# the line should be of the format %pip install PACKAGE_NAME

## **DO NOT CHANGE** the filepath variable
##### Instead, create a folder named 'data' in your current working directory and
##### have the .parquet file inside that. A relative path *must* be used when loading data into pandas

In [ ]:
# Can have as many cells as you want for code
import pandas as pd
filepath = "./data/catB_train.parquet"
# the initialised filepath MUST be a relative path to a folder named data that contains the parquet file

### **ALL** Code for machine learning and dataset analysis should be entered below.
##### Ensure that your code is clear and readable.
##### Comments and Markdown notes are advised to direct attention to pieces of code you deem useful.

In [ ]:
# import from drive
from google.colab import drive
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, f1_score, precision_score, recall_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier



from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder



pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# Mount Google Drive
drive.mount('/content/drive')
# Specify the path to your file
file_path = '/content/drive/MyDrive/Colab Notebooks/data/catB_train.parquet'

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# import dataset
df = pq.read_table(file_path).to_pandas()

#### Exploratory Data Analysis

First and foremost, we would like to pay attention to the target variable for our data analysis, which is `f_purchase_lh` in our case. For this target, something that we noted is that the target have only previous purchases recorded, and for non-purchasers the value is empty. This is something to note in our upcoming data preprocessing step.

On the other hand, the `f_purchase_lh` column is also noted to be an imbalanced columns, with a significant majority of non-purchasers. Therefore, we could frame the problem statement to be a binary classification in imbalanced data. This is important to note for our next steps.

In [ ]:
# target var - f_purchase_lh
df.f_purchase_lh.value_counts()

##### General Client Information

• clntnum: Unique identifier for the client.

As the identifier, clntnum has all unique values. It should not be seen as a feature in the final model.

• race_desc: Description of the client's race.

Based on our observation, the race of the client are mainly Chinese, Malay and Indian. However, there are up to 3996 rows of null values, which is worth noticing and paying attention to.

• ctrycode_desc: Country code indicating the client's location.

The country column stores the location from which the client is from. Based on our observations, majority of the clients are from Singapore, while 20 null values are spotted.

• clttype: Customer status.

Based on our observations, the customer status can be split into 3 major groups, namely `p`, `g`, and `c`. Majority of the customers are from `p` status, and there is no null value for the column.

• stat_flag: Flag indicating ACTIVE, LAPSED or MATURED

Majority of the clients are active clients of insurance policy, and there is no null value for the column.

• min_occ_date: Date of the client's first interaction or policy purchase with the company.

• cltdob_fix: Fixed or corrected date of birth of the client.

For min_occ_date and cltdob_fix, we do notice that there are `None` values spotted in the columns. After further researching, there are 20 and 7 `None` values in the columns respectively. This may be a sign of erroneous data.

• cltsex_fix: Fixed or corrected gender of the client.

For the gender distribution, it is quite balanced. However, there are 23 null values that we may want to pay attention to.

In [ ]:
# General Client Information

# clntnum - unique identifier
print(f'Check for uniqueness : {len(df.clntnum.unique())==len(df)}')

In [ ]:
# race_desc - race distribution of clients
print(f'Number of NA values : {df.race_desc.isna().sum()}')
race = df.race_desc.value_counts().reset_index()
plt.figure(figsize=(8, 6))
ax = sns.barplot(x='index', y='race_desc', data=race)
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')
plt.title('Client Race Distribution')
plt.xlabel('')
plt.ylabel('')
plt.show()

In [ ]:
# ctrycode_desc - client nationality distribution
print(f'Number of NA values : {df.ctrycode_desc.isna().sum()}')
country = df.ctrycode_desc.value_counts().reset_index()
plt.figure(figsize=(20, 8))
ax = sns.barplot(x='index', y='ctrycode_desc', data=country)
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')
plt.title('Client Countries Distribution')
plt.xlabel('')
plt.ylabel('')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# clttype - client type
print(f'Number of NA values : {df.clttype.isna().sum()}')
type = df.clttype.value_counts().reset_index()
plt.figure(figsize=(8, 6))
ax = sns.barplot(x='index', y='clttype', data=type)
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')
plt.title('Client Status Distribution')
plt.xlabel('')
plt.ylabel('')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# stat_flag - client status
print(f'Number of NA values : {df.stat_flag.isna().sum()}')
stat_flag = df.stat_flag.value_counts().reset_index()
plt.figure(figsize=(8, 6))
ax = sns.barplot(x='index', y='stat_flag', data=stat_flag)
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')
plt.title('Client Insurance Flag Distribution')
plt.xlabel('')
plt.ylabel('')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# cltsex_fix - client gender
print(f'Number of NA values : {df.cltsex_fix.isna().sum()}')
cltsex_fix = df.cltsex_fix.value_counts().reset_index()
plt.figure(figsize=(8, 6))
ax = sns.barplot(x='index', y='cltsex_fix', data=cltsex_fix)
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')
plt.title('Client Gender Distribution')
plt.xlabel('')
plt.ylabel('')
plt.xticks(rotation=45)
plt.show()

In [ ]:

print(f'Erroneous data {df.min_occ_date.unique().max()} is noticed in the min_ooc_date column')
print(f'Erroneous data {df.cltdob_fix.unique().max()} is noticed in the cltdob_fix column')
print(f'The earliest data is {df.min_occ_date.unique().min()} in the min_ooc_date column')
print(f'The earliest data is {df.cltdob_fix.unique().min()} is noticed in the cltdob_fix column')

##### Client Risk and Status Indicators

• flg_substandard: Flag for substandard risk clients.

• flg_is_borderline_standard: Flag for borderline standard risk clients.

• flg_is_revised_term: Flag if customer ever has revised terms.

• flg_is_rental_flat: Indicates if the client lives in a rental flat.

• flg_has_health_claim: Flag for clients with health insurance claims.

• flg_has_life_claim: Flag for clients with life insurance claims.

• flg_gi_claim: Flag for general insurance claims.

• flg_is_proposal: Indicates if there is a policy in proposal for client.

• flg_with_preauthorisation: Flag for clients with preauthorized transactions or policies.

• flg_is_returned_mail: Flag for returned mail instances.

For all the flags for certain conditions, they all have 1014 null values. Upon further checking, these NA values are from the same rows.

In [ ]:
def show_dist(col):
  val = df[f'{col}'].value_counts().reset_index()[f'{col}']
  NA = df[f'{col}'].isna().sum()
  zero = val[0]
  if len(val)>1:
    one = val[1]
  else:
    one = 0
  return zero, one, NA

In [ ]:
lsOfCols = ['flg_substandard', 'flg_is_borderline_standard', 'flg_is_revised_term', 'flg_is_rental_flat', 'flg_has_health_claim', 'flg_has_life_claim', 'flg_gi_claim', 'flg_is_proposal', 'flg_with_preauthorisation', 'flg_is_returned_mail']
flagDist = pd.DataFrame(columns=['Zero', 'One', 'NA value'])
for col in lsOfCols:
  zero, one, NA = show_dist(col)
  new_row = {'Zero': zero, 'One': one, 'NA value': NA}
  flagDist = pd.concat([flagDist, pd.DataFrame(new_row, index=[col])])
flagDist

##### Client Consent and Communication Preferences, Demographic and Household Information

• is_consent_to_mail, is_consent_to_email, is_consent_to_call, is_consent_to_sms: Flags indicating client's consent to various forms of communication.

• is_valid_dm, is_valid_email: Flags indicating the validity of direct mail and email addresses.

• is_housewife_retiree, is_sg_pr, is_class_1_2: Flags indicating specific demographics like occupation, residency status, etc.

• is_dependent_in_at_least_1_policy: Indicates if the client is a dependent in at least one policy.

For these flags, we noted the same issues as in the previous flags. Upon further checking, these NA values are from the same rows as well, which may be indicative of some underlying issues. Therefore, this may affect the rationale of filling in the null values afterwards and should be taken note.

• hh_20, pop_20, hh_size, hh_size_est: Metrics related to household size and population.

In terms of the distribution for hh_20, pop_20 and hh_size, it turns out that all these columns have similar distributions. Upon further check, the hh_size column is a function of the hh_20 column and pop_20 column. Thus, it may be wise to consider dropping those two columns as they may lead to multicollinearity between columns. The hh_size_est is just a round up of the hh_size column in front. From the observations, the majority bucket for household size is 3, while the least is 0. All 4 columns have 2809 missing values, which should be taken note in future steps.

• annual_income_est: Estimated annual income of the client, in buckets.

The annual income buckets have customers below 30k Income as the largest group. Similar to household columns, the annual income column has 2809 missing values as well, which should be taken note in future steps.

In [ ]:
lsOfCols = ['is_consent_to_mail', 'is_consent_to_email','is_consent_to_call', 'is_consent_to_sms', 'is_valid_dm','is_valid_email', 'is_housewife_retiree', 'is_sg_pr', 'is_class_1_2', 'is_dependent_in_at_least_1_policy']
flagDist = pd.DataFrame(columns=['Zero', 'One', 'NA value'])
for col in lsOfCols:
  zero, one, NA = show_dist(col)
  new_row = {'Zero': zero, 'One': one, 'NA value': NA}
  flagDist = pd.concat([flagDist, pd.DataFrame(new_row, index=[col])])
flagDist

In [ ]:
df.hh_20.dropna().astype(int).plot(kind='kde')
plt.title('hh_20 distribution')
df.hh_20.dropna().astype(int).describe()

In [ ]:
df.pop_20.dropna().astype(int).plot(kind='kde')
plt.title('Pop_20 distribution')
df.pop_20.dropna().astype(int).describe()

In [ ]:
df.hh_size.dropna().astype(int).plot(kind='kde')
plt.title('Household size distribution')
df.hh_size.dropna().astype(int).describe()

In [ ]:
print(f'Number of NA values : {df.hh_20.isna().sum()}')
print(f'Number of NA values : {df.pop_20.isna().sum()}')
print(f'Number of NA values : {df.hh_size.isna().sum()}')

In [ ]:
# hh_size_est - household size round
print(f'Number of NA values : {df.hh_size_est.isna().sum()}')
hh_size_est = df.hh_size_est.value_counts().reset_index()
plt.figure(figsize=(8, 8))
ax = sns.barplot(x='index', y='hh_size_est', data=hh_size_est)
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')
plt.title('Household Size Estimate Distribution')
plt.xlabel('')
plt.ylabel('')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# annual_income_est - household size round
print(f'Number of NA values : {df.annual_income_est.isna().sum()}')
annual_income_est = df.annual_income_est.value_counts().reset_index()
plt.figure(figsize=(8, 8))
ax = sns.barplot(x='index', y='annual_income_est', data=annual_income_est)
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')
plt.title('Annual Income Estimate Distribution')
plt.xlabel('')
plt.ylabel('')
plt.xticks(rotation=45)
plt.show()

##### Policy and Claim History

• n_months_last_bought_products, flg_latest_being_lapse, flg_latest_being_cancel, recency_lapse, recency_cancel: Metrics related to the recency of policy purchases, lapses, and cancellations.

There are up to 32 types of product from which the n months are recorded. The n_months_last_bought_products is a compilation of the user's activity in purchasing insurance in previous time steps. Thus, there may be some multicollinearity that is embedded in the columns and thus should be taken note.

• tot_inforce_pols, tot_cancel_pols: Total number of in-force and canceled policies.

• f_ever_declined_la: Flag for clients has ever been declined policies.

For columns flg_latest_being_lapse and flg_latest_being_cancel, the columns are binary flags and have no null values. These may be important indicators for the columns For other columns, these are flags for the

In [ ]:
# lsOfColnames = ["ape_lh", "prempaid_", "flg_hlthclaim_", "lapse_ape_grp", "n_months_since_lapse_", "hlthclaim_amt", "giclaim_amt", "recency_hlthclaim", "recency_giclaim", "flg_affconnect_", "f_hold", "n_months_last_bought_", "sumins_"]
lsOfColnames = ["n_months_last_bought_", "flg_latest_being_lapse", "flg_latest_being_cancel", "recency_lapse", "recency_cancel", "tot_inforce_pols", "tot_cancel_pols", "f_ever_declined_la"]
for c in lsOfColnames:
    print(f'Number of {c} columns in the dataset : {len([col for col in df.columns if (c in col)])}' )

In [ ]:
print(f'Number of NA values : {df.n_months_last_bought_products.isna().sum()}')
df.n_months_last_bought_products.dropna().astype(int).plot(kind='kde')
plt.title('n_months_last_bought_products distribution')
df.n_months_last_bought_products.dropna().astype(int).describe()

In [ ]:
# flg_latest_being_lapse distribution
print(f'Number of NA values : {df.flg_latest_being_lapse.isna().sum()}')
flg_latest_being_lapse = df.flg_latest_being_lapse.value_counts().reset_index()
plt.figure(figsize=(8, 8))
ax = sns.barplot(x='index', y='flg_latest_being_lapse', data=flg_latest_being_lapse)
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')
plt.title('flg_latest_being_lapse Distribution')
plt.xlabel('')
plt.ylabel('')
plt.show()

In [ ]:
# flg_latest_being_cancel distribution
print(f'Number of NA values : {df.flg_latest_being_cancel.isna().sum()}')
flg_latest_being_cancel = df.flg_latest_being_cancel.value_counts().reset_index()
plt.figure(figsize=(8, 6))
ax = sns.barplot(x='index', y='flg_latest_being_cancel', data=flg_latest_being_cancel)
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')
plt.title('flg_latest_being_cancel Distribution')
plt.xlabel('')
plt.ylabel('')
plt.show()

In [ ]:
print(f'Number of NA values : {df.recency_cancel.isna().sum()}')
df.recency_cancel.dropna().astype(int).plot(kind='kde')
plt.title('recency_cancel distribution')
df.recency_cancel.dropna().astype(int).describe()

In [ ]:
print(f'Number of NA values : {df.recency_lapse.isna().sum()}')
df.recency_lapse.dropna().astype(int).plot(kind='kde')
plt.title('recency_lapse distribution')
df.recency_lapse.dropna().astype(int).describe()

In [ ]:
# "tot_inforce_pols", "tot_cancel_pols", "f_ever_declined_la"
print(f'Number of NA values : {df.tot_inforce_pols.isna().sum()}')
df.tot_inforce_pols.dropna().astype(int).plot(kind='kde')
plt.title('tot_inforce_pols distribution')
df.tot_inforce_pols.dropna().astype(int).describe()

In [ ]:
print(f'Number of NA values : {df.tot_cancel_pols.isna().sum()}')
df.tot_cancel_pols.dropna().astype(int).plot(kind='kde')
plt.title('tot_cancel_pols distribution')
df.tot_cancel_pols.dropna().astype(int).describe()

In [ ]:
print(f'Number of NA values : {df.f_ever_declined_la.isna().sum()}')
f_ever_declined_la = df.f_ever_declined_la.value_counts().reset_index()
plt.figure(figsize=(8, 6))
ax = sns.barplot(x='index', y='f_ever_declined_la', data=f_ever_declined_la)
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')
plt.title('f_ever_declined_la Distribution')
plt.xlabel('')
plt.ylabel('')
plt.show()

##### Anonymized Insurance Product Metrics (APE, Sum Insured, Prepaid Premiums)

• ape_, sumins_, prempaid_* (e.g., ape_gi_42e115, sumins_ltc_1280bf, prempaid_grp_6fc3e6): Metrics for various anonymized insurance products, likely representing different types of policies like general insurance, long-term care, group policies, etc. The suffixes (like 42e115, 1280bf) are unique identifiers for the specific insurance products. ‘ape’ stands for Annual Premium Equivalent, 'sumins' for sum insured, ‘prempaid’ stands for premium customers will pay from product inception to product maturity.

Making up for the bulk of the columns in the dataset, we do notice that these columns have similar patterns and datatype. This is important to note and great news when it comes to data preprocessing steps in the future.  

##### Other Flags and Metrics

• f_elx, f_mindef_mha, f_retail: Flags possibly related to client's association with specific programs or sectors.

• flg_affconnect_*, affcon_visit_days, n_months_since_visit_affcon: Metrics related to client’s activity in affinity connect.

• clmcon_visit_days, recency_clmcon, recency_clmcon_regis: Metrics related to client’s activity in claim connect.

• hlthclaim_amt, giclaim_amt, recency_hlthclaim, recency_giclaim, hlthclaim_cnt_success, giclaim_cnt_success: Health and general insurance claim-related metrics.

• flg_hlthclaim_, flg_gi_claim_ (e.g., flg_hlthclaim_839f8a_ever, flg_gi_claim_29d435_ever): Flags for specific types of health and general insurance claims, with anonymized identifiers.

##### Purchase and Lapse Metrics for Specific Products

• f_ever_bought_, n_months_last_bought_, lapse_ape_, n_months_since_lapse_ (e.g., f_ever_bought_839f8a, n_months_last_bought_grp_6fc3e6, lapse_ape_ltc_1280bf, n_months_since_lapse_inv_dcd836): Flags and metrics indicating purchase history, lapses, and time since last interaction for various anonymized insurance products..



#### Data Preprocessing

In [ ]:
# read file
# df = pq.read_table('catB_train.parquet').to_pandas()

# filter out anomylised filters first  or ('n_months_last_bought_' in col) hlthclaim_cnt_success giclaim_cnt_success recency_clmcon_regis affcon_visit_days n_months_since_visit_affcon
anomylisedIdentifiers = [col for col in df.columns if ('ape_' in col) or ('sumins_' in col) or ('prempaid_' in col) or ('flg_hlthclaim_' in col) or ('flg_gi_claim_' in col) or ('f_ever_bought_' in col)
                                                                           or ('n_months_last_bought_' in col) or ('lapse_ape_' in col) or ('n_months_since_lapse_' in col) or ('hlthclaim_amt' in col) or ('giclaim_amt' in col) or ('recency_hlthclaim' in col)
                                                                           or ('recency_giclaim' in col)
                                                                            or ('flg_affconnect_' in col) or ('f_hold' in col)]
# generalInfo = [col for col in df.columns if col not in anomylisedIdentifiers] + ['n_months_last_bought_products']
# all columns
# ['stat_flag', 'min_occ_date', 'cltdob_fix', 'flg_substandard',
#        'flg_is_borderline_standard', 'flg_is_revised_term',
#        'flg_is_rental_flat', 'flg_has_health_claim', 'flg_has_life_claim',
#        'flg_gi_claim', 'flg_is_proposal', 'flg_with_preauthorisation',
#        'flg_is_returned_mail', 'is_consent_to_mail', 'is_consent_to_email',
#        'is_consent_to_call', 'is_consent_to_sms', 'is_valid_dm',
#        'is_valid_email', 'is_housewife_retiree', 'is_sg_pr', 'is_class_1_2',
#        'is_dependent_in_at_least_1_policy', 'f_ever_declined_la',
#        'hh_size_est', 'annual_income_est', 'flg_latest_being_lapse',
#        'flg_latest_being_cancel', 'recency_lapse', 'recency_cancel',
#        'tot_inforce_pols', 'tot_cancel_pols', 'f_elx', 'f_mindef_mha',
#        'f_retail', 'affcon_visit_days', 'n_months_since_visit_affcon',
#        'clmcon_visit_days', 'recency_clmcon', 'recency_clmcon_regis', 'race_desc', 'clttype', 'ctrycode_desc', 'cltsex_fix',
#        'stat_flag', 'annual_income_est', 'hh_size_est']

columns = ['clntnum', 'stat_flag', 'min_occ_date', 'cltdob_fix', 'flg_substandard',
       'flg_is_borderline_standard', 'flg_is_revised_term',
       'flg_is_rental_flat', 'flg_has_health_claim', 'flg_has_life_claim',
       'flg_gi_claim', 'flg_is_proposal', 'flg_with_preauthorisation',
       'flg_is_returned_mail', 'is_consent_to_mail', 'is_consent_to_email',
       'is_consent_to_call', 'is_consent_to_sms', 'is_valid_dm',
       'is_valid_email', 'is_housewife_retiree', 'is_sg_pr', 'is_class_1_2',
       'is_dependent_in_at_least_1_policy', 'f_ever_declined_la',
       'hh_size_est', 'annual_income_est', 'flg_latest_being_lapse',
       'flg_latest_being_cancel',
       'tot_inforce_pols', 'f_elx', 'f_mindef_mha',
       'f_retail', 'affcon_visit_days', 'recency_clmcon_regis', 'race_desc', 'clttype', 'ctrycode_desc', 'cltsex_fix', 'f_purchase_lh', 'hh_20', 'pop_20', 'hh_size']

################ Preprocessing for Additional Information part
additionalInfo = df[anomylisedIdentifiers + ['f_purchase_lh']]
insuranceRelated = [col for col in df.columns if ('ape_' in col) or ('sumins_' in col) or ('prempaid_' in col) or ('flg_hlthclaim_' in col) or ('flg_gi_claim_' in col) or ('f_ever_' in col)
                    or ('n_months_last_bought_' in col) or ('lapse_ape_' in col) or ('n_months_since_lapse_' in col) or ('hlthclaim_amt' in col) or ('giclaim_amt' in col) or ('recency_hlthclaim' in col)
                    or ('recency_giclaim' in col) or ('hlthclaim_cnt_success' in col) or ('giclaim_cnt_success' in col)
                    or ('flg_affconnect_' in col) or ('affcon_visit_days' in col) or ('n_months_since_visit_affcon' in col) or ('f_hold' in col)]

# preprocessing for anomylised columns
df['ape_overview'] = df[[col for col in df.columns if ('ape' in col)]].fillna(0).sum(axis=1)
for each in [col for col in df.columns if ('n_months_since_lapse_' in col)]:
    df[f'{each}'] = df[f'{each}'].apply(lambda x: 0 if x=='9999' or pd.isna(x) else float(x))
df['nmonths_overview'] = df[[col for col in df.columns if ('n_months_since_lapse_' in col)]].fillna(0).sum(axis=1)
df['prempaid_overview'] = df[[col for col in df.columns if ('prempaid_' in col)]].fillna(0).sum(axis=1)
df['fhold_overview'] = df[[col for col in df.columns if ('f_hold' in col)]].fillna(0).sum(axis=1)
df['fever_overview'] = df[[col for col in df.columns if ('f_ever' in col)]].fillna(0).sum(axis=1)
df['hlthclaim_overview'] = df[[col for col in df.columns if ('recency_hlthclaim' in col)]].fillna(0).sum(axis=1)
df['fig_overview'] = df[[col for col in df.columns if ('flg_affconnect_' in col)]].fillna(0).sum(axis=1)
df['giclaim_overview'] = df[[col for col in df.columns if ('recency_giclaim' in col)]].fillna(0).sum(axis=1)
df['sumins_overview'] = df[[col for col in df.columns if ('sumins_' in col)]].fillna(0).sum(axis=1)

df.prempaid_overview = df.prempaid_overview.astype(int).apply(lambda x: np.log(x+1))
df.ape_overview = df.ape_overview.astype(int).apply(lambda x: np.log(x+1))
# df.nmonths_overview = df.nmonths_overview.astype(int).apply(lambda x: np.log(x+1)).fillna(0) # nmonths contains negative values
df.sumins_overview = df.sumins_overview.astype(int).apply(lambda x: np.log(x+1))
additionalColumns = ['ape_overview', 'nmonths_overview', 'prempaid_overview', 'fhold_overview', 'fever_overview', 'hlthclaim_overview', 'fig_overview', 'giclaim_overview', 'sumins_overview']


In [ ]:
pca = PCA()  # Set the desired number of components
insuranceDf = pca.fit_transform(df[insuranceRelated].fillna(0))
explained_variance_ratio = pca.explained_variance_ratio_
plt.plot(explained_variance_ratio, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance Ratio vs. Number of Components')
plt.show()
cumulative_explained_variance = explained_variance_ratio.cumsum()
plt.plot(cumulative_explained_variance, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Explained Variance vs. Number of Components')
plt.show()
desired_variance_ratio = 0.95  # Set your desired explained variance ratio
n_components = np.argmax(cumulative_explained_variance >= desired_variance_ratio) + 1
print(f"Number of components to retain {desired_variance_ratio * 100}% of variance: {n_components}")

In [ ]:
additionalDf = pd.concat([df[additionalColumns].reset_index().drop('index', axis=1), pd.DataFrame(insuranceDf)], axis=1)
# merge additional columns with insurance df
#############

generalInfo = [col for col in df.columns if col not in anomylisedIdentifiers] + ['n_months_last_bought_products']
generalInfo = [col for col in generalInfo if col not in ['recency_cancel', 'tot_cancel_pols', 'clmcon_visit_days', 'recency_clmcon']]
# generalInfo = columns

# create a general copy
general = df[generalInfo].reset_index().drop('index', axis=1)
generalCopy = general.copy() # make a copy of the original dataset

# concat with the additionalDf set
general = pd.concat([general.reset_index().drop('index', axis=1), pd.DataFrame(insuranceDf)], axis=1)

# # data cleaning for general columns
# f_purchase_lh - fix purchase (target) as 0 for null value
general.f_purchase_lh = general.f_purchase_lh.fillna(0)

# race - fillna with undisclosed
general.race_desc = general.race_desc.fillna('Undisclosed')

# gender - fillna as undisclosed
general.cltsex_fix = general.cltsex_fix.fillna("Undisclosed")

# n_months_last_bought_products - fillna as 0
general.n_months_last_bought_products = general.n_months_last_bought_products.fillna(0)

# all flags - fillna as -1 to seperate from the provided values, non submission in flags may be indicative of not purchasing
# all is_ binary attributes - fillna as -1 to seperate from the provided values, non submission in any of the attributes may be indicative of not purchasing
def fillNA(value):
    general.flg_substandard = general.flg_substandard.fillna(value)
    general.flg_is_borderline_standard = general.flg_is_borderline_standard.fillna(value)
    general.flg_is_revised_term = general.flg_is_revised_term.fillna(value)
    general.flg_has_life_claim = general.flg_has_life_claim.fillna(value)
    general.flg_gi_claim = general.flg_gi_claim.fillna(value)
    general.flg_is_proposal = general.flg_is_proposal.fillna(value)
    general.flg_with_preauthorisation = general.flg_with_preauthorisation.fillna(value)
    general.flg_is_rental_flat = general.flg_is_rental_flat.fillna(value)
    general.flg_has_health_claim = general.flg_has_health_claim.fillna(value)
    general.flg_is_returned_mail = general.flg_is_returned_mail.fillna(value)

    general.is_consent_to_mail = general.is_consent_to_mail.fillna(value)
    general.is_consent_to_email = general.is_consent_to_email.fillna(value)
    general.is_consent_to_call = general.is_consent_to_call.fillna(value)
    general.is_consent_to_sms = general.is_consent_to_sms.fillna(value)
    general.is_valid_dm = general.is_valid_dm.fillna(value)
    general.is_valid_email = general.is_valid_email.fillna(value)
    general.is_housewife_retiree = general.is_housewife_retiree.fillna(value)
    general.is_sg_pr = general.is_sg_pr.fillna(value)
    general.is_class_1_2 = general.is_class_1_2.fillna(value)
    general.is_dependent_in_at_least_1_policy = general.is_dependent_in_at_least_1_policy.fillna(value)

fillNA(-1)

# giclaim_cnt_unsuccess - array of nulls, to be removed
general = general.drop('giclaim_cnt_unsuccess', axis=1)

# hlthclaim_cnt_unsuccess + hlthclaim_cnt_success - transform column into either tried to claim before or not
general.hlthclaim_cnt_unsuccess = general.hlthclaim_cnt_unsuccess.fillna(0)
general.hlthclaim_cnt_success = general.hlthclaim_cnt_success.fillna(0)
general['made_unsuccess_hlthclaim'] = general.hlthclaim_cnt_unsuccess.apply(lambda x: 1 if x!=0 else 0)
general['made_success_hlthclaim'] = general.hlthclaim_cnt_success.apply(lambda x: 1 if x!=0 else 0)
general = general.drop(['hlthclaim_cnt_unsuccess', 'hlthclaim_cnt_success'], axis=1)
general.giclaim_cnt_success = general.giclaim_cnt_success.fillna(0)
general.giclaim_cnt_success = general.giclaim_cnt_success.apply(lambda x: 1 if x!=0 else 0)
general.recency_clmcon_regis = general.recency_clmcon_regis.fillna(0)
general.recency_clmcon_regis = general.recency_clmcon_regis.apply(lambda x: 1 if x!=0 else 0)
general.affcon_visit_days = general.affcon_visit_days.fillna(0)
general.affcon_visit_days = general.affcon_visit_days.apply(lambda x: 1 if x!=0 else 0)
general.n_months_since_visit_affcon = general.n_months_since_visit_affcon.fillna(0)
general.n_months_since_visit_affcon = general.n_months_since_visit_affcon.apply(lambda x: 1 if x!=0 else 0)

# f_ever_declined_la - fill in 0 for null values
general.f_ever_declined_la = general.f_ever_declined_la.fillna(0)

# hh_size_est - keep only hh_size_est, drop hh_20, pop_20, hh_size as hh_size_est is derived from the three, fillna as -1 to differentiate from given value
general.hh_size_est = general.hh_size_est.fillna('-1')
general = general.drop(['hh_20', 'pop_20', 'hh_size'], axis=1)

# annual_income_est -  fillna as -1 to differentiate from given value
general.annual_income_est = general.annual_income_est.fillna("F-UNDISCLOSED")

# recency_lapse - fillna with median
general.recency_lapse = general.recency_lapse.fillna(general.recency_lapse.median())

# LARGE AMOUNT OF MISSING DATA - recency_cancel, tot_cancel_pols, clmcon_visit_days, recency_clmcon
# # recency_cancel - convert to binary as the amount of data missing is too much
# general.recency_cancel = general.recency_cancel.apply(lambda x : 0 if pd.isna(x) else 1)

# # tot_cancel_pols - convert to binary as the amount of data missing is too much
# general.tot_cancel_pols = general.tot_cancel_pols.apply(lambda x : 0 if pd.isna(x) else 1)

# # clmcon_visit_days, recency_clmcon - convert to binary as the amount of data missing is too much
# general.clmcon_visit_days = general.clmcon_visit_days.apply(lambda x : 0 if pd.isna(x) else 1)
# general.recency_clmcon = general.recency_clmcon.apply(lambda x : 0 if pd.isna(x) else 1)

# drop the clntnum column
general = general.drop('clntnum', axis=1)

# # preprocess text columns - clttype, ctrycode_desc, stat_flag, cltsex_fix, annual_income_est, hh_size_est
# one hot encoding for nominal columns - clttype, ctrycode_desc, cltsex_fix
encoder = OneHotEncoder()
one_hot_encoded = encoder.fit_transform(general[['race_desc', 'clttype', 'ctrycode_desc', 'cltsex_fix']])
one_hot_df = pd.DataFrame(one_hot_encoded.toarray(), columns=encoder.get_feature_names_out(['race_desc', 'clttype', 'ctrycode_desc', 'cltsex_fix']))
general = pd.concat([general, one_hot_df], axis=1)
general = general.drop(['race_desc', 'clttype', 'ctrycode_desc', 'cltsex_fix'], axis=1)

# label encoding for ordinal columns - annual_income_est, stat_flag, hh_size_est
custom_order = ['ACTIVE', 'LAPSED', 'MATURED']
ordinal_encoder = OrdinalEncoder(categories=[custom_order])
general['stat_flag'] = ordinal_encoder.fit_transform(general[['stat_flag']])

custom_order = ['F-UNDISCLOSED', 'E.BELOW30K', 'D.30K-60K', 'C.60K-100K', 'B.100K-200K', 'A.ABOVE200K']
ordinal_encoder = OrdinalEncoder(categories=[custom_order])
general['annual_income_est'] = ordinal_encoder.fit_transform(general[['annual_income_est']])

custom_order = ['-1', '0', '1', '2', '3', '4', '>4']
ordinal_encoder = OrdinalEncoder(categories=[custom_order])
general['hh_size_est'] = ordinal_encoder.fit_transform(general[['hh_size_est']])

# preprocess the datetime columns - retain delta of years for information
general.min_occ_date = general.min_occ_date.apply(lambda x : x[:4])
general.cltdob_fix = general.cltdob_fix.apply(lambda x : x[:4])

# min_occ_date -  replace with median (8.0) if null value observed
general.min_occ_date =  general.min_occ_date.replace('None', pd.NA)
general.min_occ_date = general.min_occ_date.apply(lambda x: 8.0 if pd.isna(x) else 2024 - int(x))

# for cltdob_fix - replace with median (45) if null value observed
general.cltdob_fix =  general.cltdob_fix.replace('None', pd.NA)
general.cltdob_fix = general.cltdob_fix.apply(lambda x: 45 if pd.isna(x) else 2024 - int(x))

# stratified train test split
X, y = general.drop('f_purchase_lh', axis=1), general['f_purchase_lh']
X.columns = X.columns.astype(str)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# oversampling for training set
# XtrainOversampled, ytrainOversampled = SMOTE(random_state=42).fit_resample(Xtrain, ytrain)
adasyn = ADASYN(sampling_strategy=0.5, random_state=42)
XtrainOversampled, ytrainOversampled = adasyn.fit_resample(Xtrain, ytrain)

# resampled for training
# Use RandomUnderSampler to handle class imbalance for the majority class (0)
undersampler = RandomUnderSampler(sampling_strategy=0.3, random_state=42)  # Adjust the sampling_strategy as needed
XtrainOversampled, ytrainOversampled = undersampler.fit_resample(Xtrain, ytrain)


### Model Training

#### XGBoost

In [ ]:
# Train an XGBoost classifier with regularization
xgb_model = XGBClassifier(
    learning_rate=0.1,
    scale_pos_weight=0.9,
    max_depth=3,
    min_child_weight=3,
    reg_alpha=0.41,  # L1 regularization term
    reg_lambda=1.5
)

# Fit the model on the training set
xgb_model.fit(XtrainOversampled, ytrainOversampled)

# Make predictions on the training set
y_train_pred = xgb_model.predict(XtrainOversampled)

# Calculate ROC-AUC score on the training set
roc_auc_train = roc_auc_score(ytrainOversampled, xgb_model.predict_proba(XtrainOversampled)[:, 1])

# Evaluate model performance on the training set
accuracy_train = accuracy_score(ytrainOversampled, y_train_pred)
report_train = classification_report(ytrainOversampled, y_train_pred)

# Make predictions on the test set
y_pred = xgb_model.predict(Xtest)
xgb_predProba = xgb_model.predict_proba(Xtest)[:, 1]

# Calculate ROC-AUC score on the test set
roc_auc = roc_auc_score(ytest, xgb_model.predict_proba(Xtest)[:, 1])

# Evaluate model performance on the test set
accuracy = accuracy_score(ytest, y_pred)
report = classification_report(ytest, y_pred)

# Calculate F1 score on the training set
f1_train = f1_score(ytrainOversampled, y_train_pred)

# Calculate F1 score on the test set
f1_test = f1_score(ytest, y_pred)

In [ ]:
# Print F1 scores
print("\nTraining Set F1 Score:", f1_train)
print("Test Set F1 Score:", f1_test)

In [ ]:
# Print metrics for both training and test sets
print("\nTraining Set Metrics:")
print(f"Accuracy: {accuracy_train}")
print(f"ROC-AUC: {roc_auc_train}")
print("Classification Report:\n", report_train)

print("\nTest Set Metrics:")
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print("Classification Report:\n", report)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

# Confusion matrix for training data
sns.heatmap(confusion_matrix(ytrainOversampled, y_train_pred),
            annot=True, fmt=".0f", annot_kws={"size": 18}, ax=axes[0])
axes[0].set_title('Train Data')

# Confusion matrix for test data
sns.heatmap(confusion_matrix(ytest, y_pred),
            annot=True, fmt=".0f", annot_kws={"size": 18}, ax=axes[1])
axes[1].set_title('Test Data')

plt.tight_layout()
plt.show()

In [ ]:
print("Train Data")

accuracy_train = accuracy_score(ytrainOversampled, y_train_pred)
precision_train = precision_score(ytrainOversampled, y_train_pred, average='binary')
recall_train = recall_score(ytrainOversampled, y_train_pred, average='binary')
f1_train = f1_score(ytrainOversampled, y_train_pred, average='binary')
print(f"Accuracy: {accuracy_train}")
print(f"Precision: {precision_train}")
print(f"Recall: {recall_train}")
print(f"F1 Score: {f1_train}")
print()
print("Test Data")

accuracy_test = accuracy_score(ytest, y_pred)
precision_test = precision_score(ytest, y_pred, average='binary')
recall_test = recall_score(ytest, y_pred, average='binary')
f1_test = f1_score(ytest, y_pred, average='binary')
print(f"Accuracy: {accuracy_test}")
print(f"Precision: {precision_test}")
print(f"Recall: {recall_test}")
print(f"F1 Score: {f1_test}")

#### AdaBoost

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns

base_estimator = DecisionTreeClassifier()

ada_boost_model = AdaBoostClassifier(base_estimator=base_estimator, random_state=42)

param_grid = {
    'base_estimator__max_depth': [1, 2, 3],
    'n_estimators': [20, 50, 100],
    'learning_rate': [0.01, 0.1, 1]
}

grid_search = GridSearchCV(estimator=ada_boost_model, param_grid=param_grid, cv=2, scoring='f1', verbose=1, n_jobs=-1)


grid_search.fit(XtrainOversampled, ytrainOversampled)

# Best parameters
best_ada_params = grid_search.best_params_
print("Best Parameters:", best_ada_params)

ada_model = grid_search.best_estimator_

In [ ]:
# Fit the model on the training set
ada_model.fit(XtrainOversampled, ytrainOversampled)

# Make predictions on the training set
y_train_pred = ada_model.predict(XtrainOversampled)

# Calculate ROC-AUC score on the training set
roc_auc_train = roc_auc_score(ytrainOversampled, ada_model.predict_proba(XtrainOversampled)[:, 1])

# Evaluate model performance on the training set
accuracy_train = accuracy_score(ytrainOversampled, y_train_pred)
report_train = classification_report(ytrainOversampled, y_train_pred)

# Make predictions on the test set
y_pred = ada_model.predict(Xtest)
ada_predProba = ada_model.predict_proba(Xtest)[:, 1]

# Calculate ROC-AUC score on the test set
roc_auc = roc_auc_score(ytest, ada_model.predict_proba(Xtest)[:, 1])

# Evaluate model performance on the test set
accuracy = accuracy_score(ytest, y_pred)
report = classification_report(ytest, y_pred)

# Calculate F1 score on the training set
f1_train = f1_score(ytrainOversampled, y_train_pred)

# Calculate F1 score on the test set
f1_test = f1_score(ytest, y_pred)

In [ ]:
# Print F1 scores
print("\nTraining Set F1 Score:", f1_train)
print("Test Set F1 Score:", f1_test)

In [ ]:
# Print metrics for both training and test sets
print("\nTraining Set Metrics:")
print(f"Accuracy: {accuracy_train}")
print(f"ROC-AUC: {roc_auc_train}")
print("Classification Report:\n", report_train)

print("\nTest Set Metrics:")
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print("Classification Report:\n", report)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

# Confusion matrix for training data
sns.heatmap(confusion_matrix(ytrainOversampled, y_train_pred),
            annot=True, fmt=".0f", annot_kws={"size": 18}, ax=axes[0])
axes[0].set_title('Train Data')

# Confusion matrix for test data
sns.heatmap(confusion_matrix(ytest, y_pred),
            annot=True, fmt=".0f", annot_kws={"size": 18}, ax=axes[1])
axes[1].set_title('Test Data')

plt.tight_layout()
plt.show()

In [ ]:
print("Train Data")

accuracy_train = accuracy_score(ytrainOversampled, y_train_pred)
precision_train = precision_score(ytrainOversampled, y_train_pred, average='binary')
recall_train = recall_score(ytrainOversampled, y_train_pred, average='binary')
f1_train = f1_score(ytrainOversampled, y_train_pred, average='binary')

print(f"Accuracy: {accuracy_train}")
print(f"Precision: {precision_train}")
print(f"Recall: {recall_train}")
print(f"F1 Score: {f1_train}")
print()
print("Test Data")

accuracy_test = accuracy_score(ytest, y_pred)
precision_test = precision_score(ytest, y_pred, average='binary')
recall_test = recall_score(ytest, y_pred, average='binary')
f1_test = f1_score(ytest, y_pred, average='binary')

print(f"Accuracy: {accuracy_test}")
print(f"Precision: {precision_test}")
print(f"Recall: {recall_test}")
print(f"F1 Score: {f1_test}")

#### Random Forest Classifier

In [ ]:
rf_model = RandomForestClassifier(random_state=42)

rf_param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=rf_param_grid, cv=2, scoring='f1', verbose=1, n_jobs=-1)

grid_search_rf.fit(XtrainOversampled, ytrainOversampled)

# Best parameters
best_rf_params = grid_search_rf.best_params_
print("Best Parameters for RandomForest:", best_rf_params)

# Best model
rfc_model = grid_search_rf.best_estimator_

In [ ]:
# Fit the model on the training set
rfc_model.fit(XtrainOversampled, ytrainOversampled)

# Make predictions on the training set
y_train_pred = rfc_model.predict(XtrainOversampled)

# Calculate ROC-AUC score on the training set
roc_auc_train = roc_auc_score(ytrainOversampled, rfc_model.predict_proba(XtrainOversampled)[:, 1])

# Evaluate model performance on the training set
accuracy_train = accuracy_score(ytrainOversampled, y_train_pred)
report_train = classification_report(ytrainOversampled, y_train_pred)

# Make predictions on the test set
y_pred = rfc_model.predict(Xtest)
rfc_predProba = rfc_model.predict_proba(Xtest)[:, 1]

# Calculate ROC-AUC score on the test set
roc_auc = roc_auc_score(ytest, rfc_model.predict_proba(Xtest)[:, 1])

# Evaluate model performance on the test set
accuracy = accuracy_score(ytest, y_pred)
report = classification_report(ytest, y_pred)

# Calculate F1 score on the training set
f1_train = f1_score(ytrainOversampled, y_train_pred)

# Calculate F1 score on the test set
f1_test = f1_score(ytest, y_pred)

In [ ]:
# Print F1 scores
print("\nTraining Set F1 Score:", f1_train)
print("Test Set F1 Score:", f1_test)

In [ ]:
# Print metrics for both training and test sets
print("\nTraining Set Metrics:")
print(f"Accuracy: {accuracy_train}")
print(f"ROC-AUC: {roc_auc_train}")
print("Classification Report:\n", report_train)

print("\nTest Set Metrics:")
print(f"Accuracy: {accuracy}")
print(f"ROC-AUC: {roc_auc}")
print("Classification Report:\n", report)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

# Confusion matrix for training data
sns.heatmap(confusion_matrix(ytrainOversampled, y_train_pred),
            annot=True, fmt=".0f", annot_kws={"size": 18}, ax=axes[0])
axes[0].set_title('Train Data')

# Confusion matrix for test data
sns.heatmap(confusion_matrix(ytest, y_pred),
            annot=True, fmt=".0f", annot_kws={"size": 18}, ax=axes[1])
axes[1].set_title('Test Data')

plt.tight_layout()
plt.show()

In [ ]:
print("Train Data")

accuracy_train = accuracy_score(ytrainOversampled, y_train_pred)
precision_train = precision_score(ytrainOversampled, y_train_pred, average='binary')
recall_train = recall_score(ytrainOversampled, y_train_pred, average='binary')
f1_train = f1_score(ytrainOversampled, y_train_pred, average='binary')

print(f"Accuracy: {accuracy_train}")
print(f"Precision: {precision_train}")
print(f"Recall: {recall_train}")
print(f"F1 Score: {f1_train}")
print()
print("Test Data")

accuracy_test = accuracy_score(ytest, y_pred)
precision_test = precision_score(ytest, y_pred, average='binary')
recall_test = recall_score(ytest, y_pred, average='binary')
f1_test = f1_score(ytest, y_pred, average='binary')

print(f"Accuracy: {accuracy_test}")
print(f"Precision: {precision_test}")
print(f"Recall: {recall_test}")
print(f"F1 Score: {f1_test}")

#### Model Stacking

In [ ]:
# model stacking
final_predProba = (rfc_predProba + xgb_predProba + ada_predProba)/3

# optimise for threshold
best_threshold = 0
max_score = 0
for t in range(30):
    threshold = 0.4 + t*0.01
    final_pred = [1 if p>threshold else 0 for p in final_predProba]
    f1 = f1_score(ytest, final_pred, average='binary')
    accuracy = accuracy_score(ytest, final_pred)
    score = round(f1*0.8, 4) + round(accuracy*0.2, 4) # adjust weightage round(f1, 4)
    if score>max_score:
        max_score = score
        best_threshold = threshold
final_pred = [1 if p>best_threshold else 0 for p in final_predProba]
print(f'The best threshold is {best_threshold}')

accuracy = accuracy_score(ytest, final_pred)
precision = precision_score(ytest, final_pred, average='binary')
recall = recall_score(ytest, final_pred, average='binary')
f1 = f1_score(ytest, final_pred, average='binary')
sns.heatmap(confusion_matrix(ytest, final_pred), annot=True)
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

## The cell below is **NOT** to be removed
##### The function is to be amended so that it accepts the given input (dataframe) and returns the required output (list).
##### It is recommended to test the function out prior to submission
-------------------------------------------------------------------------------------------------------------------------------
##### The hidden_data parsed into the function below will have the same layout columns wise as the dataset *SENT* to you
##### Thus, ensure that steps taken to modify the initial dataset to fit into the model are also carried out in the function below

In [ ]:
def testing_hidden_data(hidden_data: pd.DataFrame) -> list:
    df = hidden_data
    # filter out anomylised filters first  or ('n_months_last_bought_' in col) hlthclaim_cnt_success giclaim_cnt_success recency_clmcon_regis affcon_visit_days n_months_since_visit_affcon
    anomylisedIdentifiers = [col for col in df.columns if ('ape_' in col) or ('sumins_' in col) or ('prempaid_' in col) or ('flg_hlthclaim_' in col) or ('flg_gi_claim_' in col) or ('f_ever_bought_' in col)
                                                                           or ('n_months_last_bought_' in col) or ('lapse_ape_' in col) or ('n_months_since_lapse_' in col) or ('hlthclaim_amt' in col) or ('giclaim_amt' in col) or ('recency_hlthclaim' in col)
                                                                           or ('recency_giclaim' in col)
                                                                            or ('flg_affconnect_' in col) or ('f_hold' in col)]

    insuranceRelated = [col for col in df.columns if ('ape_' in col) or ('sumins_' in col) or ('prempaid_' in col) or ('flg_hlthclaim_' in col) or ('flg_gi_claim_' in col) or ('f_ever_' in col)
                    or ('n_months_last_bought_' in col) or ('lapse_ape_' in col) or ('n_months_since_lapse_' in col) or ('hlthclaim_amt' in col) or ('giclaim_amt' in col) or ('recency_hlthclaim' in col)
                    or ('recency_giclaim' in col) or ('hlthclaim_cnt_success' in col) or ('giclaim_cnt_success' in col)
                    or ('flg_affconnect_' in col) or ('affcon_visit_days' in col) or ('n_months_since_visit_affcon' in col) or ('f_hold' in col)]

    # preprocessing for anomylised columns
    df['ape_overview'] = df[[col for col in df.columns if ('ape' in col)]].fillna(0).sum(axis=1)
    for each in [col for col in df.columns if ('n_months_since_lapse_' in col)]:
        df[f'{each}'] = df[f'{each}'].apply(lambda x: 0 if x=='9999' or pd.isna(x) else float(x))
    df['nmonths_overview'] = df[[col for col in df.columns if ('n_months_since_lapse_' in col)]].fillna(0).sum(axis=1)
    df['prempaid_overview'] = df[[col for col in df.columns if ('prempaid_' in col)]].fillna(0).sum(axis=1)
    df['fhold_overview'] = df[[col for col in df.columns if ('f_hold' in col)]].fillna(0).sum(axis=1)
    df['fever_overview'] = df[[col for col in df.columns if ('f_ever' in col)]].fillna(0).sum(axis=1)
    df['hlthclaim_overview'] = df[[col for col in df.columns if ('recency_hlthclaim' in col)]].fillna(0).sum(axis=1)
    df['fig_overview'] = df[[col for col in df.columns if ('flg_affconnect_' in col)]].fillna(0).sum(axis=1)
    df['giclaim_overview'] = df[[col for col in df.columns if ('recency_giclaim' in col)]].fillna(0).sum(axis=1)
    df['sumins_overview'] = df[[col for col in df.columns if ('sumins_' in col)]].fillna(0).sum(axis=1)

    df.prempaid_overview = df.prempaid_overview.astype(int).apply(lambda x: np.log(x+1))
    df.ape_overview = df.ape_overview.astype(int).apply(lambda x: np.log(x+1))
    df.sumins_overview = df.sumins_overview.astype(int).apply(lambda x: np.log(x+1))
    additionalColumns = ['ape_overview', 'nmonths_overview', 'prempaid_overview', 'fhold_overview', 'fever_overview', 'hlthclaim_overview', 'fig_overview', 'giclaim_overview', 'sumins_overview']

    pca = PCA(n_components=7)
    insuranceDf = pca.fit_transform(df[insuranceRelated].fillna(0))
    additionalDf = pd.concat([df[additionalColumns].reset_index().drop('index', axis=1), pd.DataFrame(insuranceDf)], axis=1)

    generalInfo = [col for col in df.columns if col not in anomylisedIdentifiers] + ['n_months_last_bought_products']
    generalInfo = [col for col in generalInfo if col not in ['recency_cancel', 'tot_cancel_pols', 'clmcon_visit_days', 'recency_clmcon']]
    # create a general copy
    general = df[generalInfo].reset_index().drop('index', axis=1)
    generalCopy = general.copy() # make a copy of the original dataset

    # concat with the additionalDf set
    general = pd.concat([general.reset_index().drop('index', axis=1), pd.DataFrame(insuranceDf)], axis=1)
    # # data cleaning for general columns
    # f_purchase_lh - fix purchase (target) as 0 for null value
    general.f_purchase_lh = general.f_purchase_lh.fillna(0)

    # race - fillna with undisclosed
    general.race_desc = general.race_desc.fillna('Undisclosed')

    # gender - fillna as undisclosed
    general.cltsex_fix = general.cltsex_fix.fillna("Undisclosed")

    # n_months_last_bought_products - fillna as
    general.n_months_last_bought_products = general.n_months_last_bought_products.fillna(0)

    # all flags - fillna as -1 to seperate from the provided values, non submission in flags may be indicative of not purchasin
    # all is_ binary attributes - fillna as -1 to seperate from the provided values, non submission in any of the attributes may be indicative of not purchasing
    def fillNA(value):
        general.flg_substandard = general.flg_substandard.fillna(value)
        general.flg_is_borderline_standard = general.flg_is_borderline_standard.fillna(value)
        general.flg_is_revised_term = general.flg_is_revised_term.fillna(value)
        general.flg_has_life_claim = general.flg_has_life_claim.fillna(value)
        general.flg_gi_claim = general.flg_gi_claim.fillna(value)
        general.flg_is_proposal = general.flg_is_proposal.fillna(value)
        general.flg_with_preauthorisation = general.flg_with_preauthorisation.fillna(value)
        general.flg_is_rental_flat = general.flg_is_rental_flat.fillna(value)
        general.flg_has_health_claim = general.flg_has_health_claim.fillna(value)
        general.flg_is_returned_mail = general.flg_is_returned_mail.fillna(value)

        general.is_consent_to_mail = general.is_consent_to_mail.fillna(value)
        general.is_consent_to_email = general.is_consent_to_email.fillna(value)
        general.is_consent_to_call = general.is_consent_to_call.fillna(value)
        general.is_consent_to_sms = general.is_consent_to_sms.fillna(value)
        general.is_valid_dm = general.is_valid_dm.fillna(value)
        general.is_valid_email = general.is_valid_email.fillna(value)
        general.is_housewife_retiree = general.is_housewife_retiree.fillna(value)
        general.is_sg_pr = general.is_sg_pr.fillna(value)
        general.is_class_1_2 = general.is_class_1_2.fillna(value)
        general.is_dependent_in_at_least_1_policy = general.is_dependent_in_at_least_1_policy.fillna(value)
    fillNA(-1)

    # giclaim_cnt_unsuccess - array of nulls, to be remove
    general = general.drop('giclaim_cnt_unsuccess', axis=1)
    # hlthclaim_cnt_unsuccess + hlthclaim_cnt_success - transform column into either tried to claim before or not
    general.hlthclaim_cnt_unsuccess = general.hlthclaim_cnt_unsuccess.fillna(0)
    general.hlthclaim_cnt_success = general.hlthclaim_cnt_success.fillna(0)
    general['made_unsuccess_hlthclaim'] = general.hlthclaim_cnt_unsuccess.apply(lambda x: 1 if x!=0 else 0)
    general['made_success_hlthclaim'] = general.hlthclaim_cnt_success.apply(lambda x: 1 if x!=0 else 0)
    general = general.drop(['hlthclaim_cnt_unsuccess', 'hlthclaim_cnt_success'], axis=1)
    general.giclaim_cnt_success = general.giclaim_cnt_success.fillna(0)
    general.giclaim_cnt_success = general.giclaim_cnt_success.apply(lambda x: 1 if x!=0 else 0)
    general.recency_clmcon_regis = general.recency_clmcon_regis.fillna(0)
    general.recency_clmcon_regis = general.recency_clmcon_regis.apply(lambda x: 1 if x!=0 else 0)
    general.affcon_visit_days = general.affcon_visit_days.fillna(0)
    general.affcon_visit_days = general.affcon_visit_days.apply(lambda x: 1 if x!=0 else 0)
    general.n_months_since_visit_affcon = general.n_months_since_visit_affcon.fillna(0)
    general.n_months_since_visit_affcon = general.n_months_since_visit_affcon.apply(lambda x: 1 if x!=0 else 0)

    # f_ever_declined_la - fill in 0 for null values
    general.f_ever_declined_la = general.f_ever_declined_la.fillna(0)

    # hh_size_est - keep only hh_size_est, drop hh_20, pop_20, hh_size as hh_size_est is derived from the three, fillna as -1 to differentiate from given value
    general.hh_size_est = general.hh_size_est.fillna('-1')
    general = general.drop(['hh_20', 'pop_20', 'hh_size'], axis=1)

    # annual_income_est -  fillna as -1 to differentiate from given value
    general.annual_income_est = general.annual_income_est.fillna("F-UNDISCLOSED")

    # recency_lapse - fillna with median
    general.recency_lapse = general.recency_lapse.fillna(general.recency_lapse.median())

    # drop the clntnum column
    general = general.drop('clntnum', axis=1)

    # # preprocess text columns - clttype, ctrycode_desc, stat_flag, cltsex_fix, annual_income_est, hh_size_est
    # one hot encoding for nominal columns - clttype, ctrycode_desc, cltsex_fix
    encoder = OneHotEncoder()
    one_hot_encoded = encoder.fit_transform(general[['race_desc', 'clttype', 'ctrycode_desc', 'cltsex_fix']])
    one_hot_df = pd.DataFrame(one_hot_encoded.toarray(), columns=encoder.get_feature_names_out(['race_desc', 'clttype', 'ctrycode_desc', 'cltsex_fix']))
    general = pd.concat([general, one_hot_df], axis=1)
    general = general.drop(['race_desc', 'clttype', 'ctrycode_desc', 'cltsex_fix'], axis=1)

    # label encoding for ordinal columns - annual_income_est, stat_flag, hh_size_est
    custom_order = ['ACTIVE', 'LAPSED', 'MATURED']
    ordinal_encoder = OrdinalEncoder(categories=[custom_order])
    general['stat_flag'] = ordinal_encoder.fit_transform(general[['stat_flag']])

    custom_order = ['F-UNDISCLOSED', 'E.BELOW30K', 'D.30K-60K', 'C.60K-100K', 'B.100K-200K', 'A.ABOVE200K']
    ordinal_encoder = OrdinalEncoder(categories=[custom_order])
    general['annual_income_est'] = ordinal_encoder.fit_transform(general[['annual_income_est']])
    custom_order = ['-1', '0', '1', '2', '3', '4', '>4']
    ordinal_encoder = OrdinalEncoder(categories=[custom_order])
    general['hh_size_est'] = ordinal_encoder.fit_transform(general[['hh_size_est']])

    # preprocess the datetime columns - retain delta of years for information
    general.min_occ_date = general.min_occ_date.apply(lambda x : x[:4])
    general.cltdob_fix = general.cltdob_fix.apply(lambda x : x[:4])

    # min_occ_date -  replace with median (8.0) if null value observed
    general.min_occ_date =  general.min_occ_date.replace('None', pd.NA)
    general.min_occ_date = general.min_occ_date.apply(lambda x: 8.0 if pd.isna(x) else 2024 - int(x))

    # for cltdob_fix - replace with median (45) if null value observed
    general.cltdob_fix =  general.cltdob_fix.replace('None', pd.NA)
    general.cltdob_fix = general.cltdob_fix.apply(lambda x: 45 if pd.isna(x) else 2024 - int(x))

    rfc_pred_proba = rfc_model.predict_proba(general)[:, 1]
    xgb_pred_proba = xgb_model.predict_proba(general)[:, 1]
    ada_pred_proba = ada_model.predict_proba(general)[:, 1]

    final_pred_proba = (rfc_pred_proba + xgb_pred_proba + ada_pred_proba) / 3

    # Determine final predictions using the best_threshold
    result = [1 if p > best_threshold else 0 for p in final_pred_proba]

    return general

##### Cell to check testing_hidden_data function

In [ ]:
# This cell should output a list of predictions.
test_df = pd.read_parquet(filepath)
test_df = test_df.drop(columns=["f_purchase_lh"])
print(testing_hidden_data(test_df))

### Please have the filename renamed and ensure that it can be run with the requirements above being met. All the best!